<a href="https://colab.research.google.com/github/xiaochengJF/DeepLearning/blob/master/Faster_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font face=STCAIYUN size=10 color=purple>构建Faster RCNN</font>
<font face=楷体 color=skyblue>

- Region Proposal network (RPN)  
- RPN loss functions  
- Region of Interest Pooling (ROI)  
- ROI loss functions
</font>

**<font face=楷体 color=skyblue size=5>训练Faster RCNN时通常的数据流</font>**

<font face=楷体>1、从图像中提取特征  
2、产生anchor目标  
3、RPN网络中得到位置和目标预测分值  
4、取前N个坐标及其目标得分即建议层  
5、传递前N个坐标通过Fast R-CNN网络，生成4中建议的每个位置的位置和cls预测  
6、对4中建议的每个坐标生成建议目标  
7、采用2,3计算rpn_cls_loss和rpn_reg_loss  
8、采用5,6计算roi_cls_loss和roi_reg_loss  
</font>  
<font face=楷体 size=5 color=green>绿色链接：</font>  
【1】[Guide to build Faster RCNN in PyTorch](https://medium.com/@fractaldle/guide-to-build-faster-rcnn-in-pytorch-95b10c273439)

<font face=楷体 color=skyblue>定义一张图像、两个bbox及其标签</font>

In [0]:
import torch
image = torch.zeros((1, 3, 800, 800)).float()

bbox = torch.FloatTensor([[20, 30, 400, 500], [300, 400, 500, 600]]) # [y1, x1, y2, x2] format
labels = torch.LongTensor([6, 8]) # 0 represents background
sub_sample = 16

<font face=楷体 color=skyblue>生成一个dummy image并且设置volatile为False</font>



In [2]:
import torchvision
dummy_img = torch.zeros((1, 3, 800, 800)).float()
print(dummy_img.shape)
#Out: torch.Size([1, 3, 800, 800])

torch.Size([1, 3, 800, 800])


<font face=楷体 color=skyblue>列出VGG16的所有层</font>

In [3]:
# model = torchvision.models.vgg16(pretrained=True)
model = torchvision.models.vgg16(pretrained=False)
fe = list(model.features)
# print(fe) # length is 15
for layer in fe:
    print (layer)

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True

<font face=楷体 color=skyblue>将图像喂给网络，确定得到相应的输出尺寸</font>

In [4]:
req_features = []
k = dummy_img.clone()
for i in fe:
    k = i(k)
    if k.size()[2] < 800//16:
        break
    req_features.append(i)
    out_channels = k.size()[1]
print(len(req_features)) #30
print(out_channels) # 512

30
512


<font face=楷体 color=skyblue>将list转换为Sequential module</font>

In [0]:
faster_rcnn_fe_extractor = torch.nn.Sequential(*req_features)

<font face=楷体 color=skyblue>用构建好的VGG16网络提取特征图</font>

In [6]:
out_map = faster_rcnn_fe_extractor(image)
print(out_map.size())

torch.Size([1, 512, 50, 50])


## Anchor boxes
<font face=楷体>

- 在所有feature map的坐标上生成Anchor
- 对每个目标分配标签及坐标（相对于anchor）
- 在feature map坐标生成Anchor  

采用anchor_scales=[8，16，32] ratio=[0.5，1，2] subsampling=16
</font>

<font face=楷体 color=skyblue>feature map的一个像素位置生成9个anchor boxes，每个像素对应输入图像中的16*16像素</font>

In [7]:
import numpy as np
ratios = [0.5, 1, 2]
anchor_scales = [8, 16, 32]

anchor_base = np.zeros((len(ratios) * len(anchor_scales), 4), dtype=np.float32)

print(anchor_base)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [8]:
ctr_y = sub_sample / 2.
ctr_x = sub_sample / 2.
print(ctr_y, ctr_x)

for i in range(len(ratios)):
 for j in range(len(anchor_scales)):
   h = sub_sample * anchor_scales[j] * np.sqrt(ratios[i])    # 输出的feature map的每个像素对应图像中的16*16像素
   w = sub_sample * anchor_scales[j] * np.sqrt(1./ ratios[i])  # ration决定了anchor的长宽比例(同面积)

   index = i * len(anchor_scales) + j

   anchor_base[index, 0] = ctr_y - h / 2.
   anchor_base[index, 1] = ctr_x - w / 2.
   anchor_base[index, 2] = ctr_y + h / 2.
   anchor_base[index, 3] = ctr_x + w / 2.
print(anchor_base)

8.0 8.0
[[ -37.254833  -82.50967    53.254833   98.50967 ]
 [ -82.50967  -173.01933    98.50967   189.01933 ]
 [-173.01933  -354.03867   189.01933   370.03867 ]
 [ -56.        -56.         72.         72.      ]
 [-120.       -120.        136.        136.      ]
 [-248.       -248.        264.        264.      ]
 [ -82.50967   -37.254833   98.50967    53.254833]
 [-173.01933   -82.50967   189.01933    98.50967 ]
 [-354.03867  -173.01933   370.03867   189.01933 ]]


<font face=楷体 color=skyblue>feature map的每一个像素位置都要生成9个anchor boxes，每个像素对应输入图像中的16*16像素</font>

In [9]:
fe_size = (800//16)
ctr_x = np.arange(16, (fe_size+1) * 16, 16)
ctr_y = np.arange(16, (fe_size+1) * 16, 16)
print (ctr_x)

[ 16  32  48  64  80  96 112 128 144 160 176 192 208 224 240 256 272 288
 304 320 336 352 368 384 400 416 432 448 464 480 496 512 528 544 560 576
 592 608 624 640 656 672 688 704 720 736 752 768 784 800]


<font face=楷体 color=skyblue>将feature map每个像素位置作为锚点并映射回输入图像中</font>

In [0]:
index = 0
ctr  = np.zeros(shape=(len(ctr_x)*len(ctr_y),2))
for x in range(len(ctr_x)):
   for y in range(len(ctr_y)):
       ctr[index, 1] = ctr_x[x] - 8  # 前面生成中心点位置偏移了8个像素点
       ctr[index, 0] = ctr_y[y] - 8
       index +=1

In [12]:
anchors = np.zeros(shape=(fe_size * fe_size * 9, 4))  # 共有2500个锚点，每个锚点9个框，每个框4个参数
index = 0
for c in ctr:
 ctr_y, ctr_x = c
 for i in range(len(ratios)):
   for j in range(len(anchor_scales)):
     h = sub_sample * anchor_scales[j] * np.sqrt(ratios[i])
     w = sub_sample * anchor_scales[j] * np.sqrt(1./ ratios[i])
     anchors[index, 0] = ctr_y - h / 2.
     anchors[index, 1] = ctr_x - w / 2.
     anchors[index, 2] = ctr_y + h / 2.
     anchors[index, 3] = ctr_x + w / 2.
     index += 1
print(anchors.shape)
#Out: [22500, 4]

(22500, 4)


<font face=楷体 color=skyblue size=5>给anchor分配标签和位置</font>  
<font face=楷体>
a)与ground-truth-box IoU最大的anchor标记为正标签  
b)与ground-truth-box IoU大于0.7的anchor标记为正标签  
c)与ground-truth-box IoU小于0.3的anchor标记为负标签  
d)其余anchor既不是正样本的也不是负样本，对训练没有帮助  
<font color=red>注意：</font>单个ground-truth对象可以为多个anchor分配正标签  
</font>
<font face=楷体 color=skyblue>通过如下方式对anchor boxes分配标签和位置：</font>   
<font face=楷体>
1、找到有效的anchor boxes的索引，并且生成索引数组，生成标签数组其形状索引数组填充-1（无效anchor boxes，对应上文说的处在边框外的anchor boxes）。  
2、检查是否满足以上a、b、c条件中的一条，并相应填写标签。如果是正anchor box(标签为1)，注意哪个ground-truth目标可以得到这个结果。  
3、计算与anchor box相关的ground-truth的位置(loc)。  
4、通过为所有无效的anchor box填充-1和为所有有效锚箱计算的值来重新组织所有anchor box。  
5、输出应该是(N, 1)数组的标签和带有(N, 4)数组的locs。  
6、找到所有有效anchor boxes的索引：
</font>

In [0]:
bbox = np.asarray([[20, 30, 400, 500], [300, 400, 500, 600]], dtype=np.float32) # [y1, x1, y2, x2] format
labels = np.asarray([6, 8], dtype=np.int8) # 0 represents background 

<font face=楷体>去除坐标出界的边框，保留图片内的框</font>

In [13]:
inside_index = np.where(
       (anchors[:, 0] >= 0) &
       (anchors[:, 1] >= 0) &
       (anchors[:, 2] <= 800) &
       (anchors[:, 3] <= 800)
   )[0]
print(inside_index.shape)
#Out: (8940,)

(8940,)


<font face=楷体>初始化标签为-1</font>

In [27]:
label = np.empty((len(inside_index), ), dtype=np.int32)
label.fill(-1)
print(label.shape)
#Out = (8940, )

(8940,)


<font face=楷体>获取有效anchor（图片内anchor）坐标</font>

In [14]:
valid_anchor_boxes = anchors[inside_index]
print(valid_anchor_boxes.shape)
#Out = (8940, 4)

(8940, 4)


<font face=楷体>计算每个anchor框与所有目标框的IOU，这里目标框共2个</font>

In [15]:
ious = np.empty((len(valid_anchor_boxes), 2), dtype=np.float32)
ious.fill(0)
print("两个目标框为：\n{}".format(bbox))
for num1, i in enumerate(valid_anchor_boxes):
   ya1, xa1, ya2, xa2 = i 
   anchor_area = (ya2 - ya1) * (xa2 - xa1)  # 有效框面积
   for num2, j in enumerate(bbox):
       yb1, xb1, yb2, xb2 = j
       box_area = (yb2- yb1) * (xb2 - xb1)  # 目标框面积

       inter_x1 = max([xb1, xa1])
       inter_y1 = max([yb1, ya1])
       inter_x2 = min([xb2, xa2])
       inter_y2 = min([yb2, ya2])
       if (inter_x1 < inter_x2) and (inter_y1 < inter_y2):
           iter_area = (inter_y2 - inter_y1) * \
(inter_x2 - inter_x1)
           iou = iter_area / \
(anchor_area + box_area - iter_area) 
       else:
           iou = 0.

       ious[num1, num2] = iou
print(ious.shape)

两个目标框为：
tensor([[ 20.,  30., 400., 500.],
        [300., 400., 500., 600.]])
(8940, 2)


<font face=楷体 color=skyblue>分别找到与每个gt_box iou最高的 anchor box(两个)</font> 

In [16]:
gt_argmax_ious = ious.argmax(axis=0)
print(gt_argmax_ious)
gt_max_ious = ious[gt_argmax_ious, np.arange(ious.shape[1])]
print(gt_max_ious)
# Out:
# [2262 5620]
# [0.68130493 0.61035156]

[2262 5620]
[0.68130493 0.61035156]


<font face=楷体 color=skyblue>得到每个anchor box与所有ground-truth box中 的最高iou</font> 


In [20]:
argmax_ious = ious.argmax(axis=1)
print(argmax_ious.shape)
print(argmax_ious)
max_ious = ious[np.arange(len(inside_index)), argmax_ious]
print(max_ious.shape)
print(max_ious)
# Out:
# (22500,)
# [0, 1, 0, ..., 1, 0, 0]
# [0.06811669 0.07083762 0.07083762 ... 0.         0.         0.        ]

(8940,)
[0 0 0 ... 0 0 0]
(8940,)
[0.06811669 0.07083762 0.07083762 ... 0.         0.         0.        ]


<font face=楷体>根据上面获取的目标最大IOU值，获取等于该值的index</font>

In [24]:
gt_argmax_ious = np.where(ious == gt_max_ious)[0]
print(ious[np.where(ious == gt_max_ious)])
print(gt_argmax_ious)
# Out:
# [2262, 2508, 5620, 5628, 5636, 5644, 5866, 5874, 5882, 5890, 6112,
#        6120, 6128, 6136, 6358, 6366, 6374, 6382]

[0.68130493 0.68130493 0.61035156 0.61035156 0.61035156 0.61035156
 0.61035156 0.61035156 0.61035156 0.61035156 0.61035156 0.61035156
 0.61035156 0.61035156 0.61035156 0.61035156 0.61035156 0.61035156]
[2262 2508 5620 5628 5636 5644 5866 5874 5882 5890 6112 6120 6128 6136
 6358 6366 6374 6382]


<font face=楷体>设置正负样本阈值</font>

In [0]:
pos_iou_threshold  = 0.7
neg_iou_threshold = 0.3

label[max_ious < neg_iou_threshold] = 0  # 小于0.3为负样本
label[gt_argmax_ious] = 1  # 最大IOU对应anchor为正样本
label[max_ious >= pos_iou_threshold] = 1  # 大于0.7为正

<font face=楷体>设置正负样本比例及总量<font>

In [0]:
pos_ratio = 0.5  # 正样本比例
n_sample = 256  # 样本总量

n_pos = pos_ratio * n_sample  # 正样本数量

<font face=楷体>如果正样本数量大于n_pos，则随机抽取n_pos个正样本</font>

In [38]:
pos_index = np.where(label == 1)[0]
print(np.where(label == 1))
print(pos_index)
if len(pos_index) > n_pos:
   disable_index = np.random.choice(pos_index, size=(len(pos_index) - n_pos), replace=False)
   label[disable_index] = -1

(array([2262, 2508, 5620, 5628, 5636, 5644, 5866, 5874, 5882, 5890, 6112,
       6120, 6128, 6136, 6358, 6366, 6374, 6382]),)
[2262 2508 5620 5628 5636 5644 5866 5874 5882 5890 6112 6120 6128 6136
 6358 6366 6374 6382]


In [0]:
n_neg = n_sample * np.sum(label == 1)
neg_index = np.where(label == 0)[0]
if len(neg_index) > n_neg:
   disable_index = np.random.choice(neg_index, size=(len(neg_index) - n_neg), replace = False)
   label[disable_index] = -1

In [42]:
print(bbox)  
print(argmax_ious)
max_iou_bbox = bbox[argmax_ious]
print(max_iou_bbox)
#Out
# [[ 20.,  30., 400., 500.],
#  [ 20.,  30., 400., 500.],
#  [ 20.,  30., 400., 500.],
#  ...,
#  [ 20.,  30., 400., 500.],
#  [ 20.,  30., 400., 500.],
#  [ 20.,  30., 400., 500.]]

tensor([[ 20.,  30., 400., 500.],
        [300., 400., 500., 600.]])
[0 0 0 ... 0 0 0]
tensor([[ 20.,  30., 400., 500.],
        [ 20.,  30., 400., 500.],
        [ 20.,  30., 400., 500.],
        ...,
        [ 20.,  30., 400., 500.],
        [ 20.,  30., 400., 500.],
        [ 20.,  30., 400., 500.]])


<font face=楷体>有效anchor的中心点和宽高：ctr_x, ctr_y, width, height  
有效anchor对应目标框的中心点和宽高: base_ctr_x, base_ctr_y, base_width, base_height</font>

In [0]:
height = valid_anchor_boxes[:, 2] - valid_anchor_boxes[:, 0]
width = valid_anchor_boxes[:, 3] - valid_anchor_boxes[:, 1]
ctr_y = valid_anchor_boxes[:, 0] + 0.5 * height
ctr_x = valid_anchor_boxes[:, 1] + 0.5 * width

base_height = max_iou_bbox[:, 2] - max_iou_bbox[:, 0]
base_width = max_iou_bbox[:, 3] - max_iou_bbox[:, 1]
base_ctr_y = max_iou_bbox[:, 0] + 0.5 * base_height
base_ctr_x = max_iou_bbox[:, 1] + 0.5 * base_width

<font face=楷体 color=red>有效anchor转为目标框的系数（dy，dx是平移系数；dh，dw是缩放系数）  
【1】[python numpy np.finfo()函数 eps](https://blog.csdn.net/Dontla/article/details/103062246)</font>

In [44]:
eps = np.finfo(height.dtype).eps  # 代表非负的最小值
height = np.maximum(height, eps)
width = np.maximum(width, eps)
dy = (base_ctr_y - ctr_y) / height
dx = (base_ctr_x - ctr_x) / width
dh = np.log(base_height / height)
dw = np.log(base_width / width)
anchor_locs = np.vstack((dy, dx, dh, dw)).transpose()
print(anchor_locs)
#Out:
# [[ 0.5855727   2.3091455   0.7415673   1.647276  ]
#  [ 0.49718437  2.3091455   0.7415673   1.647276  ]
#  [ 0.40879607  2.3091455   0.7415673   1.647276  ]
#  ...
#  [-2.50802    -5.292254    0.7415677   1.6472763 ]
#  [-2.5964084  -5.292254    0.7415677   1.6472763 ]
#  [-2.6847968  -5.292254    0.7415677   1.6472763 ]]

[[ 0.5855728   2.30914558  0.7415674   1.64727602]
 [ 0.49718446  2.30914558  0.7415674   1.64727602]
 [ 0.40879611  2.30914558  0.7415674   1.64727602]
 ...
 [-2.50801936 -5.29225232  0.7415674   1.64727602]
 [-2.59640771 -5.29225232  0.7415674   1.64727602]
 [-2.68479606 -5.29225232  0.7415674   1.64727602]]


<font face=楷体>每个anchor框对应的label（-1：无效anchor，0：负有效anchor，1：正有效anchor）</font>

In [0]:
anchor_labels = np.empty((len(anchors),), dtype=label.dtype)
anchor_labels.fill(-1)
anchor_labels[inside_index] = label

In [0]:
anchor_locations = np.empty((len(anchors),) + anchors.shape[1:], dtype=anchor_locs.dtype)
anchor_locations.fill(0)
anchor_locations[inside_index, :] = anchor_locs
print(anchor_locs.shape)
print(anchor_locations.shape)

(8940, 4)
(22500, 4)


In [0]:
import torch.nn as nn
mid_channels = 512
in_channels = 512 # depends on the output feature map. in vgg 16 it is equal to 512
n_anchor = 9 # Number of anchors at each location
conv1 = nn.Conv2d(in_channels, mid_channels, 3, 1, 1)
reg_layer = nn.Conv2d(mid_channels, n_anchor *4, 1, 1, 0)
cls_layer = nn.Conv2d(mid_channels, n_anchor *2, 1, 1, 0) ## I will be going to use softmax here. you can equally use sigmoid if u replace 2 with 1.

In [0]:
# conv sliding layer
conv1.weight.data.normal_(0, 0.01)
conv1.bias.data.zero_()

# Regression layer
reg_layer.weight.data.normal_(0, 0.01)
reg_layer.bias.data.zero_()

# classification layer
cls_layer.weight.data.normal_(0, 0.01)
cls_layer.bias.data.zero_()

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
x = conv1(out_map) # out_map is obtained in section 1
pred_anchor_locs = reg_layer(x)
pred_cls_scores = cls_layer(x)

print(pred_cls_scores.shape, pred_anchor_locs.shape)

#Out:
#torch.Size([1, 18, 50, 50]) torch.Size([1, 36, 50, 50])

(torch.Size([1, 18, 50, 50]), torch.Size([1, 36, 50, 50]))


In [0]:
pred_anchor_locs = pred_anchor_locs.permute(0, 2, 3, 1).contiguous().view(1, -1, 4)
print(pred_anchor_locs.shape)

#Out: torch.Size([1, 22500, 4])

pred_cls_scores = pred_cls_scores.permute(0, 2, 3, 1).contiguous()
print(pred_cls_scores.shape)
#Out torch.Size([1, 50, 50, 18])

objectness_score = pred_cls_scores.view(1, 50, 50, 9, 2)[:, :, :, :, 1].contiguous().view(1, -1)
print(objectness_score.shape)
#Out torch.Size([1, 22500])

pred_cls_scores  = pred_cls_scores.view(1, -1, 2)
print(pred_cls_scores.shape)
# Out torch.size([1, 22500, 2])

torch.Size([1, 22500, 4])
torch.Size([1, 50, 50, 18])
torch.Size([1, 22500])
torch.Size([1, 22500, 2])


In [0]:
nms_thresh = 0.7
n_train_pre_nms = 12000
n_train_post_nms = 2000
n_test_pre_nms = 6000
n_test_post_nms = 300
min_size = 16

In [0]:
anc_height = anchors[:, 2] - anchors[:, 0]
anc_width = anchors[:, 3] - anchors[:, 1]
anc_ctr_y = anchors[:, 0] + 0.5 * anc_height
anc_ctr_x = anchors[:, 1] + 0.5 * anc_width

In [0]:
pred_anchor_locs_numpy = pred_anchor_locs[0].data.numpy()
objectness_score_numpy = objectness_score[0].data.numpy()

dy = pred_anchor_locs_numpy[:, 0::4]
dx = pred_anchor_locs_numpy[:, 1::4]
dh = pred_anchor_locs_numpy[:, 2::4]
dw = pred_anchor_locs_numpy[:, 3::4]

ctr_y = dy * anc_height[:, np.newaxis] + anc_ctr_y[:, np.newaxis]
ctr_x = dx * anc_width[:, np.newaxis] + anc_ctr_x[:, np.newaxis]
h = np.exp(dh) * anc_height[:, np.newaxis]
w = np.exp(dw) * anc_width[:, np.newaxis]

In [0]:
roi = np.zeros(pred_anchor_locs_numpy.shape, dtype=anchor_locs.dtype)
roi[:, 0::4] = ctr_y - 0.5 * h
roi[:, 1::4] = ctr_x - 0.5 * w
roi[:, 2::4] = ctr_y + 0.5 * h
roi[:, 3::4] = ctr_x + 0.5 * w
print(roi)
#Out:
# [[ -36.897102,  -80.29519 ,   54.09939 ,  100.40507 ],
#  [ -83.12463 , -165.74298 ,   98.67854 ,  188.6116  ],
#  [-170.7821  , -378.22214 ,  196.20844 ,  349.81198 ],
#  ...,
#  [ 696.17816 ,  747.13306 ,  883.4582  ,  836.77747 ],
#  [ 621.42114 ,  703.0614  ,  973.04626 ,  885.31226 ],
#  [ 432.86267 ,  622.48926 , 1146.7059  ,  982.9209  ]]

[[ -36.67576154  -79.27511906   51.64738999  100.11247437]
 [ -83.70975854 -172.16819977   95.69221742  186.44289897]
 [-181.49092789 -353.39055923  194.17979089  367.47368701]
 ...
 [ 707.64184898  748.10783793  885.99900389  837.87469098]
 [ 621.36531166  701.43897911  973.33845843  877.71796126]
 [ 430.30310964  604.16962506 1164.04136903  970.94035367]]


In [0]:
img_size = (800, 800) #Image size
roi[:, slice(0, 4, 2)] = np.clip(
           roi[:, slice(0, 4, 2)], 0, img_size[0])
roi[:, slice(1, 4, 2)] = np.clip(
   roi[:, slice(1, 4, 2)], 0, img_size[1])

print(roi)

#Out:
# [[  0.     ,   0.     ,  54.09939, 100.40507],
#  [  0.     ,   0.     ,  98.67854, 188.6116 ],
#  [  0.     ,   0.     , 196.20844, 349.81198],
#  ...,
#  [696.17816, 747.13306, 800.     , 800.     ],
#  [621.42114, 703.0614 , 800.     , 800.     ],
#  [432.86267, 622.48926, 800.     , 800.     ]]

[[  0.           0.          51.64738999 100.11247437]
 [  0.           0.          95.69221742 186.44289897]
 [  0.           0.         194.17979089 367.47368701]
 ...
 [707.64184898 748.10783793 800.         800.        ]
 [621.36531166 701.43897911 800.         800.        ]
 [430.30310964 604.16962506 800.         800.        ]]


In [0]:
hs = roi[:, 2] - roi[:, 0]
ws = roi[:, 3] - roi[:, 1]
keep = np.where((hs >= min_size) & (ws >= min_size))[0]
roi = roi[keep, :]
score = objectness_score_numpy[keep]

print(score.shape)
#Out:
##(22500, ) all the boxes have minimum size of 16

(22500,)


In [0]:
order = score.ravel().argsort()[::-1]
print(order)

#Out:
#[ 889,  929, 1316, ...,  462,  454,    4]

[ 457 1335  898 ...  893  884  461]


In [0]:
type(order)

numpy.ndarray

In [0]:
order = order[:n_train_pre_nms]
roi = roi[order, :]

print(roi.shape)
print(roi)

#Out
# (12000, 4)
# [[607.93866,   0.     , 800.     , 113.38187],
#  [  0.     ,   0.     , 235.29704, 369.64795],
#  [572.177  ,   0.     , 800.     , 373.0086 ],
#  ...,
#  [250.07968, 186.61633, 434.6356 , 276.70615],
#  [490.07974, 154.6163 , 674.6356 , 244.70615],
#  [266.07968, 602.61633, 450.6356 , 692.7062 ]]

(12000, 4)
[[  0.           0.         192.55664604 111.93685307]
 [710.39430714   0.         800.         104.54028177]
 [623.83468338   0.         800.         110.45556365]
 ...
 [337.94808388 200.30745438 800.         711.63463333]
 [113.94808388 376.30745438 626.82851601 800.        ]
 [257.94808388 200.30745438 770.82851601 711.63463333]]


In [0]:
anchor_area = (ya2 - ya1) * (xa2 - xa1)
   for num2, j in enumerate(bbox):
       yb1, xb1, yb2, xb2 = j
       box_area = (yb2- yb1) * (xb2 - xb1)

IndentationError: ignored

In [0]:
y1 = roi[:, 0]
x1 = roi[:, 1]
y2 = roi[:, 2]
x2 = roi[:, 3]

areas = (x2 - x1 + 1) * (y2 - y1 + 1)
print(order.shape)
order = score.argsort()[::-1]  
########################################
O = []
for o in order:
    if o < 12000:
        O.append(0)
order = np.asarray(O)
print(order.shape)
########################################
keep = []

while order.size > 0:
   i = order[0]
   xx1 = np.maximum(x1[i], x1[order[1:]])
   yy1 = np.maximum(y1[i], y1[order[1:]])
   xx2 = np.minimum(x2[i], x2[order[1:]])
   yy2 = np.minimum(y2[i], y2[order[1:]])
  
   w = np.maximum(0.0, xx2 - xx1 + 1)
   h = np.maximum(0.0, yy2 - yy1 + 1)    
   inter = w * h
   ovr = inter / (areas[i] + areas[order[1:]] - inter)
   
   inds = np.where(ovr <= nms_thresh)[0]
   order = order[inds + 1]

keep = keep[:n_train_post_nms] # while training/testing , use accordingly
roi = roi[keep] # the final region proposals

(12000,)
(12000,)


<font face=楷体>n_sample：roi中采样的样本数目，默认为128  
pos_ratio：n_samples中的正样本的比例，默认为0.25   
pos_iou_thresh：设置为正样本region proposal与ground-truth目标之间最小的重叠值阈值  
[neg_iou_threshold_lo, neg_iou_threshold_hi] : [0.0, 0.5], 设置为负样本[背景]的重叠值阈值  

In [0]:
n_sample = 128
pos_ratio = 0.25
pos_iou_thresh = 0.5
neg_iou_thresh_hi = 0.5
neg_iou_thresh_lo = 0.0

<font face=楷体 color=skyblue>找到每个ground-truth 目标与region proposal的iou</font> 


In [0]:
ious = np.empty((len(roi), 2), dtype=np.float32)
ious.fill(0)
for num1, i in enumerate(roi):
   ya1, xa1, ya2, xa2 = i  
   anchor_area = (ya2 - ya1) * (xa2 - xa1)
   for num2, j in enumerate(bbox):
       yb1, xb1, yb2, xb2 = j
       box_area = (yb2- yb1) * (xb2 - xb1)

       inter_x1 = max([xb1, xa1])
       inter_y1 = max([yb1, ya1])
       inter_x2 = min([xb2, xa2])
       inter_y2 = min([yb2, ya2])

       if (inter_x1 < inter_x2) and (inter_y1 < inter_y2):
           iter_area = (inter_y2 - inter_y1) * \
(inter_x2 - inter_x1)
           iou = iter_area / (anchor_area+ \
box_area - iter_area)            
       else:
           iou = 0.

       ious[num1, num2] = iou
print(ious.shape)

#Out:
#[1535, 2]

(0, 2)



<font face=楷体 color=skyblue>找到与每个region proposal具有较高IoU的ground truth，并且找到最大的IoU</font> 

In [0]:
gt_assignment = iou.argmax(axis=1)
max_iou = iou.max(axis=1)
print(gt_assignment)
print(max_iou)

#Out:
# [0, 0, 0 ... 1, 1, 0]
# [0.016, 0., 0. ... 0.08034518, 0.10739268, 0.]

AttributeError: ignored

<font face=楷体 color=skyblue>为每个proposal分配标签</font>   
<font face=楷体><font face=楷体 color=red>注意：</font>这里默认背景标记为0</font>

In [0]:
gt_roi_label = labels[gt_assignment]
print(gt_roi_label)
#Out:
#[6, 6, 6, ..., 8, 8, 6] q  

NameError: ignored

<font face=楷体>
根据每个pos_iou_thresh选择前景rois。只保留n_sample*pos_ratio（128*0.25=32）个前景样本，因此如果只得到少于32个正样本，保持原状。如果得到多余32个前景目标，从中采样32个样本
</font>

In [0]:
pos_index = np.where(max_iou >= pos_iou_thresh)[0]
pos_roi_per_this_image = int(min(pos_roi_per_image, pos_index.size))
if pos_index.size > 0:
   pos_index = np.random.choice(
       pos_index, size=pos_roi_per_this_image, replace=False)
print(pos_roi_per_this_image)
print(pos_index)

#Out
# 18
# [ 257  296  317 1075 1077 1169 1213 1258 1322 1325 1351 1378 1380 1425
#  1472 1482 1489 1495]


NameError: ignored

<font face=楷体>
针对负[背景]region proposal进行相似处理，如果对于之前分配的ground truth目标，region proposal的IoU在neg_iou_thresh_lo和neg_iou_thresh_hi之间，对该region proposal分配0标签，从这些负样本中采样n(n_sample-pos_samples,128-32=96)个region proposals
</font>

In [0]:
neg_index = np.where((max_iou < neg_iou_thresh_hi) &
                            (max_iou >= neg_iou_thresh_lo))[0]
neg_roi_per_this_image = n_sample - pos_roi_per_this_image
neg_roi_per_this_image = int(min(neg_roi_per_this_image,
                                neg_index.size))
if  neg_index.size > 0 :
   neg_index = np.random.choice(
       neg_index, size=neg_roi_per_this_image, replace=False)
print(neg_roi_per_this_image)
print(neg_index)

#Out:
#110
# [  79  688  160  ...  376  712 1235  148 1001]

<font face=楷体>整合正样本索引和负样本索引，及他们各自的标签和region proposals</font>

In [0]:
keep_index = np.append(pos_index, neg_index)
gt_roi_labels = gt_roi_label[keep_index]
gt_roi_labels[pos_roi_per_this_image:] = 0  # negative labels --> 0
sample_roi = roi[keep_index]
print(sample_roi.shape)

#Out:
#(128, 4)

<font face=楷体>对这些sample_roi选择ground truth目标之后按照第二节中为anchor boxes分配位置的方式进行参数化</font>
$$\begin{matrix}
&t_{x} = (x - x_{a})/w_{a} 
&t_{y} = (y - y_{a})/h_{a}\\
&t_{w} = log(w/ w_a)
&t_{h} = log(h/ h_a)
\end{matrix}$$

In [0]:
bbox_for_sampled_roi = bbox[gt_assignment[keep_index]]
print(bbox_for_sampled_roi.shape)
#Out
#(128, 4)
height = sample_roi[:, 2] - sample_roi[:, 0]
width = sample_roi[:, 3] - sample_roi[:, 1]
ctr_y = sample_roi[:, 0] + 0.5 * height
ctr_x = sample_roi[:, 1] + 0.5 * width
base_height = bbox_for_sampled_roi[:, 2] - bbox_for_sampled_roi[:, 0]
base_width = bbox_for_sampled_roi[:, 3] - bbox_for_sampled_roi[:, 1]
base_ctr_y = bbox_for_sampled_roi[:, 0] + 0.5 * base_height
base_ctr_x = bbox_for_sampled_roi[:, 1] + 0.5 * base_width

In [0]:
#t_{x} = (x - x_{a})/w_{a}
#t_{y} = (y - y_{a})/h_{a}
#t_{w} = log(w/ w_a)
#t_{h} = log(h/ h_a)

eps = np.finfo(height.dtype).eps
height = np.maximum(height, eps)
width = np.maximum(width, eps)

dy = (base_ctr_y - ctr_y) / height
dx = (base_ctr_x - ctr_x) / width
dh = np.log(base_height / height)
dw = np.log(base_width / width)

gt_roi_locs = np.vstack((dy, dx, dh, dw)).transpose()
print(gt_roi_locs)

#Out:
# [[-0.08075945, -0.14638858, -0.23822695, -0.23150307],
#  [ 0.04865225,  0.15570255,  0.08902431, -0.5969549 ],
#  [ 0.17411101,  0.2244332 ,  0.19870323,  0.25063717],
#  .....
#  [-0.13976236,  0.121031  ,  0.03863466,  0.09662855],
#  [-0.59361845, -2.5121436 ,  0.04558792,  0.9731178 ],
#  [ 0.1041566 , -0.7840459 ,  1.4283055 ,  0.95092565]]

In [0]:
rois = torch.from_numpy(sample_rois).float()
roi_indices = 0 * np.ones((len(rois),), dtype=np.int32)
roi_indices = torch.from_numpy(roi_indices).float()
print(rois.shape, roi_indices.shape)
#Out:
#torch.Size([128, 4]) torch.Size([128])

NameError: ignored

合并 rois and roi_indices, 这样我们将会得到维度是[N, 5] (index, x, y, h, w)的张量

In [0]:
indices_and_rois = torch.cat([roi_indices[:, None], rois], dim=1)
xy_indices_and_rois = indices_and_rois[:, [0, 2, 1, 4, 3]]
indices_and_rois = xy_indices_and_rois.contiguous()
print(xy_indices_and_rois.shape)
#Out:
#torch.Size([128, 5])

要将数组传到roi_pooling层,定义大小为 7 x 7

In [0]:
size = (7, 7)
adaptive_max_pool = AdaptiveMaxPool2d(size[0], size[1])
output = []
rois = indices_and_rois.data.float()
rois[:, 1:].mul_(1/16.0) # Subsampling ratio
rois = rois.long()
num_rois = rois.size(0)
for i in range(num_rois):
   roi = rois[i]
   im_idx = roi[0]
   im = out_map.narrow(0, im_idx, 1)[..., roi[2]:(roi[4]+1), roi[1]:(roi[3]+1)]
   output.append(adaptive_max_pool(im))
output = torch.cat(output, 0)
print(output.size())
#Out:
# torch.Size([128, 512, 7, 7])
# Reshape the tensor so that we can pass it through the feed forward layer.
k = output.view(output.size(0), -1)
print(k.shape)
#Out:
# torch.Size([128, 25088])

这将会是一个classifier层的输入, 进一步将会如同下面图表所示的分出classification head 和 regression head 。 现在让我们定义网络

In [0]:
roi_head_classifier = nn.Sequential(*[nn.Linear(25088, 4096),
                                     nn.Linear(4096, 4096)])
cls_loc = nn.Linear(4096, 21 * 4) # (VOC 20 classes + 1 background. Each wil
cls_loc.weight.data.normal_(0, 0.01)
cls_loc.bias.data.zero_()
score = nn.Linear(4096, 21) # (VOC 20 classes + 1 background)

将roi-pooling的输出传到上面我们定义的网络

In [0]:
k = roi_head_classifier(k)
roi_cls_loc = cls_loc(k)
roi_cls_score = score(k)
print(roi_cls_loc.shape, roi_cls_score.shape)
#Out:
# torch.Size([128, 84]), torch.Size([128, 21])

roi_cls_loc 和 roi_cls_score 是从实际边界区域得到的两个输出张量

## 损失函数
<font face=楷体>
其中$p_{i}$是预测的班级标签，$p_{i}^*$是实际的班级分数。$t_{i}$并且$t_{i}^*$是预测的原点和实际的坐标。$p_{i}^*$如果锚点为正，则地面真实标签为1；如果锚点为负，则地面真实标签为0。我们将在Pytorch中看到如何完成此操作    
RPN和Fast-RCNN都含有两种损失Regression和 classification
</font>

In [0]:
print(pred_anchor_locs.shape)
print(pred_cls_scores.shape)
print(anchor_locations.shape)
print(anchor_labels.shape)
#Out:
# torch.Size([1, 12321, 4])
# torch.Size([1, 12321, 2])
# (12321, 4)
# (12321,)

将输入和输出排成一行

In [0]:
rpn_loc = pred_anchor_locs[0]
rpn_score = pred_cls_scores[0]
gt_rpn_loc = torch.from_numpy(anchor_locations)
gt_rpn_score = torch.from_numpy(anchor_labels)
print(rpn_loc.shape, rpn_score.shape, gt_rpn_loc.shape, gt_rpn_score.shape)
#Out
# torch.Size([12321, 4]) torch.Size([12321, 2]) torch.Size([12321, 4])

pred_cls_scores 和 anchor_labels 是RPN网络的预测对象值和实际对象值  
对classification用Cross Entropy损失:
$$H(y)=(-1)*\sum_iy_i*log(y_i)\\
Softmax = e^{l_{ina}}/(\sum^3_{a=1}e^{O_{ina}})$$

用Pytorch计算损失

In [0]:
import torch.nn.functional as F
rpn_cls_loss = F.cross_entropy(rpn_score, gt_rpn_score.long(), ignore_index = -1)
print(rpn_cls_loss)
#Out:
# Variable containing:
#  0.6940
# [torch.FloatTensor of size 1]

对于 Regression 用smooth L1 损失:
$$\begin{aligned}
L_{Loc}(t^u,v)&=\sum_{i\in x,y,w,h}smooth_{L_1}(t^u_i-V_i)\\
smooth_{L_1} &=\left\{ \begin{aligned} 0.5*x^2 \quad if |x|<1\\
|x|-0.5 \quad otherwise\end{aligned}\right.
\end{aligned}$$

使用 L1 而不是 L2 损失，是因为RPN的预测回归头的值不是有限的。 Regression 损失也被应用在有正标签的边界区域中：

In [0]:
pos = gt_rpn_score > 0
mask = pos.unsqueeze(1).expand_as(rpn_loc)
print(mask.shape)
#Out:
# torch.Size(12321, 4)

取有正数标签的边界区域：

In [0]:
mask_loc_preds = rpn_loc[mask].view(-1, 4)
mask_loc_targets = gt_rpn_loc[mask].view(-1, 4)
print(mask_loc_preds.shape, mask_loc_preds.shape)
#Out:
# torch.Size([6, 4]) torch.Size([6, 4])

regression损失应用如下

In [0]:
x = torch.abs(mask_loc_targets - mask_loc_preds)
rpn_loc_loss = ((x < 1).float() * 0.5 * x**2) + ((x >= 1).float() * (x-0.5))
print(rpn_loc_loss.sum())
#Out:
# Variable containing:
#  0.3826
# [torch.FloatTensor of size 1]

合并rpn_cls_loss 和 rpn_reg_loss, 因为class loss 应用在全部的边界区域，regression loss 应用在正数标签边界区域,作者已经介绍 Λ 作为超参数。通过使用边界区域的数量：

In [0]:
rpn_lambda = 10.
N_reg = (gt_rpn_score >0).float().sum()
rpn_loc_loss = rpn_loc_loss.sum() / N_reg
rpn_loss = rpn_cls_loss + (rpn_lambda * rpn_loc_loss)
print(rpn_loss)
#Out:0.00248

Fast RCNN 损失

预测

In [0]:
print(roi_cls_loc.shape)
print(roi_cls_score.shape)
#Out:
# torch.Size([128, 84])
# torch.Size([128, 21])

真实

In [0]:
print(gt_roi_locs.shape)
print(gt_roi_labels.shape)
#Out:
#(128, 4)
#(128, )

转化到Torch变量

In [0]:
gt_roi_loc = torch.from_numpy(gt_roi_locs)
gt_roi_label = torch.from_numpy(np.float32(gt_roi_labels)).long()
print(gt_roi_loc.shape, gt_roi_label.shape)
#Out:
#torch.Size([128, 4]) torch.Size([128])

分类损失

In [0]:
roi_clss_loss = F.cross_entropy(roi_cls_score, rt_roi_label, ignore_index=-1)
print(roi_cls_loss.shape)
#Out:
#Variable containing:
#  3.0458
# [torch.FloatTensor of size 1]

对于回归损失，每个ROI位置有21（num_classes+background）预测边界框。为了计算损失，我们将只使用带有正标签的边界框（P_i^*）

In [0]:
n_sample = roi_cls_loc.shape[0]
roi_loc = roi_cls_loc.view(n_sample, -1, 4)
print(roi_loc.shape)
#Out:
#torch.Size([128, 21, 4])
roi_loc = roi_loc[torch.arange(0, n_sample).long(), gt_roi_label]
print(roi_loc.shape)
#Out:
#torch.Size([128, 4])

用计算RPN网络回归损失的方法计算回归损失
注意，我们这里没有写任何regloss函数。读者可以包装在RPN Reg Loss中讨论的所有方法，并实现这个函数

In [0]:
roi_loc_loss = REGLoss(roi_loc, gt_roi_loc)
print(roi_loc_loss)
#Out:
#Variable containing:
#  0.1895
# [torch.FloatTensor of size 1]

ROI损失总和

In [0]:
roi_lambda = 10.
roi_loss = roi_cls_loss + (roi_lambda * roi_loc_loss)
print(roi_loss)
#Out:
#Variable containing:
#  4.2353
# [torch.FloatTensor of size 1]

损失总和
RPN损失+ Fast RCNN损失

In [0]:
total_loss = rpn_loss + roi_loss

In [0]:
!wget https://github.com/liuyuemaicha/simple_faster_rcnn.git

--2020-02-09 02:17:01--  https://github.com/liuyuemaicha/simple_faster_rcnn.git
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/liuyuemaicha/simple_faster_rcnn [following]
--2020-02-09 02:17:01--  https://github.com/liuyuemaicha/simple_faster_rcnn
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘simple_faster_rcnn.git’

simple_faster_rcnn.     [ <=>                ]  76.76K  --.-KB/s    in 0.007s  

2020-02-09 02:17:02 (10.0 MB/s) - ‘simple_faster_rcnn.git’ saved [78607]



In [0]:
#coding:utf8

import torch
import torchvision
from PIL import Image, ImageDraw
import numpy as np
from model import VGG
import utils


# 大体流程：
# 1. 图像通过vgg获得特征图，
# 2. 特征图通过RPN获得有效anchor的置信度(foreground)和转为预测框的坐标系数
# 3. 特征图和预测框通过ROI Pooling获取固定尺寸的预测目标特征图,即利用预测框，从特征图中把目标抠出来，
#    因为目标尺寸不一，再通过ROI Pooling的方法把目标转为统一的固定尺寸（7*7），这样就可以方便做目标的分类和预测框的修正处理。
# 4. 固定尺寸的预测目标特征图通过类别分类模型（self.score）获取预测框所属的类别，
# 5. 固定尺寸的预测目标特征图通过坐标分类模型(self.cls_loc)获取置信度和预测框修正的坐标系数。

# 假设 图片中的两个目标框"ground-truth"
bbox = np.asarray([[20, 30, 400, 500], [300, 400, 500, 600]], dtype=np.float32) # [y1, x1, y2, x2] format
# 假设 图片中两个目标框分别对应的标签
labels = np.asarray([6, 8], dtype=np.int8)  # 0 represents background

img_tensor = torch.zeros((1, 3, 800, 800)).float()
img_var = torch.autograd.Variable(img_tensor)


# ---------------------step_1: 获取目标anchor的置信度（anchor_conf）和平移缩放系数（anchor_locations）
# 初始化所有anchors, 并找出有效anchors和对应的index
# anchors： (22500, 4)  valid_anchor_boxes： (8940, 4)  valid_anchor_index：8940
anchors, valid_anchor_boxes, valid_anchor_index = utils.init_anchor()
# 计算有效anchors与所有目标框的IOU
# ious：（8940, 2) 每个有效anchor框与目标实体框的IOU
ious = utils.compute_iou(valid_anchor_boxes, bbox)
valid_anchor_len = len(valid_anchor_boxes)
# 在有效框中找到一定比例的正例和负例
label, argmax_ious = utils.get_pos_neg_sample(ious, valid_anchor_len, pos_iou_threshold=0.7,
                                 neg_iou_threshold=0.3, pos_ratio=0.5, n_sample=256)
# print np.sum(label == 1)  # 18个正例
# print np.sum(label == 0)  # 256-18=238个负例

# 现在让我们用具有最大iou的ground truth对象为每个anchor box分配位置。
# 注意，我们将为所有有效的anchor box分配anchor locs，而不考虑其标签，稍后在计算损失时，我们可以使用简单的过滤器删除它们。
# 每个有效anchor对应的目标框bbox
max_iou_bbox = bbox[argmax_ious]  # 有效anchor框对应的目标框坐标  (8940, 4)
# print max_iou_bbox.shape  # (8940, 4)，共有8940个有效anchor框，每个anchor有坐标值（y1, x1, y2, x2）
# 为所有有效的anchor_box分配anchor_locs，anchor_locs是每个有效的anchors转为对应目标框（bbox）的平移缩放系数
anchor_locs = utils.get_coefficient(valid_anchor_boxes, max_iou_bbox)
# print(anchor_locs.shape)  # (8940, 4)  4维参数（平移参数：dy, dx； 缩放参数：dh, dw）

# anchor_conf ： 所有anchor框对应的label（-1：无效anchor，0：负例有效anchor，1：正例有效anchor）
anchor_conf = np.empty((len(anchors),), dtype=label.dtype)
anchor_conf.fill(-1)
anchor_conf[valid_anchor_index] = label
print anchor_conf.shape  # 所有anchor对应的label（feature_size*feature_size*9）=》 (22500,)

# anchor_locations： 所有anchor框转为目标实体框的系数，无效anchor系数全部为0，有效anchor有有效系数
anchor_locations = np.empty((len(anchors),) + anchors.shape[1:], dtype=anchor_locs.dtype)
anchor_locations.fill(0)
anchor_locations[valid_anchor_index, :] = anchor_locs
print anchor_locations.shape  # 所有anchor对应的平移缩放系数（feature_size*feature_size*9，4）=》(22500, 4)

# 这里通过候选anchor与目标实体框计算得到anchor框的置信度（anchor_conf）和平移缩放系数（anchor_locations）
# ----------------------


# --------------------step_2: VGG 和 RPN 模型: RPN 预测的是anchor转为目标框的平移缩放系数
vgg = VGG()
# out_map 特征图， # pred_anchor_locs 预测anchor框到目标框转化的系数， pred_anchor_conf 预测anchor框的分数
out_map, pred_anchor_locs, pred_anchor_conf = vgg.forward(img_var)
print out_map.data.shape  # (batch_size, num, feature_size, feature_size) => (1, 512, 50, 50)

# 1. pred_anchor_locs 预测每个anchor框到目标框转化的系数（平移缩放），与 anchor_locations对应
pred_anchor_locs = pred_anchor_locs.permute(0, 2, 3, 1).contiguous().view(1, -1, 4)
print(pred_anchor_locs.shape)  # Out: torch.Size([1, 22500, 4])

# 2. 预测anchor框的置信度，每个anchor框都会对应一个置信度，与 anchor_conf对应
pred_anchor_conf = pred_anchor_conf.permute(0, 2, 3, 1).contiguous()
print(pred_anchor_conf.shape)  # Out torch.Size([1, 50, 50, 18])
objectness_score = pred_anchor_conf.view(1, 50, 50, 9, 2)[:, :, :, :, 1].contiguous().view(1, -1)
print(objectness_score.shape)  # Out torch.Size([1, 22500])

pred_anchor_conf = pred_anchor_conf.view(1, -1, 2)
print(pred_anchor_conf.shape)  # Out torch.size([1, 22500, 2])
# ---------------------


# ---------------------step_3: RPN 损失 （有效anchor与预测anchor之间的损失--坐标系数损失与置信度损失）
# 从上面step_1中，我们得到了目标anchor信息：
# 目标anchor坐标系数：anchor_locations  (22500, 4)
# 目标anchor置信度：anchor_conf  (22500,)

# 从上面step_2中，我们得到了预测anchor信息：
# RPN网络预测anchor的坐标系数：pred_anchor_locs  (1, 22500, 4)
# RPN网络预测anchor的置信度: pred_anchor_conf  (1, 22500, 2)

# 我们将会从新排列，将输入和输出排成一行
rpn_anchor_loc = pred_anchor_locs[0]
rpn_anchor_conf = pred_anchor_conf[0]
anchor_locations = torch.from_numpy(anchor_locations)
anchor_conf = torch.from_numpy(anchor_conf)
print(rpn_anchor_loc.shape, rpn_anchor_conf.shape, anchor_locations.shape, anchor_conf.shape)
# torch.Size([22500, 4]) torch.Size([22500, 2]) torch.Size([22500, 4]) torch.Size([22500])

rpn_loss = vgg.roi_loss(rpn_anchor_loc, rpn_anchor_conf, anchor_locations, anchor_conf, weight=10.0)
print("rpn_loss: {}".format(rpn_loss))  # 1.33919
# ---------------------


# ---------------------step_4: 根据anchor和预测anchor系数，计算预测框（roi）和预测框的坐标系数(roi_locs)，
# ---------------------并得到每个预测框的所属类别label(roi_labels)
# 通过anchors框和模型预测的平移缩放系数，得到预测框ROI；再通过预测的分值和阈值进行过滤精简
roi, score, order = utils.get_predict_bbox(anchors, pred_anchor_locs, objectness_score,
                                           n_train_pre_nms=12000, min_size=16)

# 得到的预测框（ROI）还会有大量重叠，再通过NMS（非极大抑制）做进一步的过滤精简
roi = utils.nms(roi, score, order, nms_thresh=0.7, n_train_post_nms=2000)


# 根据预测框ROI与目标框BBox的IOU，得到每个预测框所要预测的目标框（预测框与哪个目标框的IOU大，就代表预测哪个目标）；
# 并根据IOU对ROI做进一步过滤，并划分正负样例。
sample_roi, keep_index, gt_assignment, roi_labels = utils.get_propose_target(roi, bbox, labels,
                                                                                n_sample=128,
                                                                                pos_ratio=0.25,
                                                                                pos_iou_thresh=0.5,
                                                                                neg_iou_thresh_hi=0.5,
                                                                                neg_iou_thresh_lo=0.0)
# print(sample_roi.shape)  # (128, 4)
# 预测框对应的目标框 bbox_for_sampled_roi
bbox_for_sampled_roi = bbox[gt_assignment[keep_index]]  # 目标框
print(bbox_for_sampled_roi.shape)  # (128, 4)
# 预测框（ROI）转目标框的真实系数
roi_locs = utils.get_coefficient(sample_roi, bbox_for_sampled_roi)
# ---------------------


# ---------------------step_5: ROI Pooling：
# 这一步做了两件事：
# 一是从特征图中根据ROI把相应的预测目标框抠出来(im)
# 二是将抠出来的预测目标框通过adaptive_max_pool方法，输出为固定尺寸(512, 7, 7)，方便后续的批处理
# 这样的特点：
# 一是并没有在输入图像上预测，而是在VGG模型的输出特征图上进行预测，这样减少了计算量；
# 二是因为目标实体尺寸多种多样，通过ROI Pooling方法将输出统一为固定尺寸(512, 7, 7)，方便进行批处理，
# sample_roi：预测的有效框 (128, 4)
rois = torch.from_numpy(sample_roi).float()
# roi_indices：添加图像的索引[这里我们只有一个图像，其索引号为0]
roi_indices = 0 * np.ones((len(rois),), dtype=np.int32)
roi_indices = torch.from_numpy(roi_indices).float()
print(rois.shape, roi_indices.shape)  # torch.Size([128, 4]) torch.Size([128])

# 将图像的索引号和预测的有效框进行合并, 这样我们将会得到维度是[N, 5]  5=>(index, x1, y1, x2, y2)的张量
indices_and_rois = torch.cat([roi_indices[:, None], rois], dim=1)  # torch.Size([128, 5])

output = []
rois = indices_and_rois.float()
rois[:, 1:].mul_(1/16.0)  # 对预测框进行下采样，匹配特征图out_map
rois = rois.long()
num_rois = rois.size(0)
# out_map: (batch_size, num, feature_size, feature_size) => (1, 512, 50, 50)
for i in range(num_rois):
   roi = rois[i]
   im_idx = roi[0]  # 图片的索引号
   # 取出索引号是im_idx的图片特征图=》(1, 512, 50, 50)，因为本实例就一张图片，所以操作完后shape并不变
   out_map = out_map.narrow(0, im_idx, 1)
   # 这一步是根据预测框的的x1,y1, x2,y2坐标，从特征图out_map中把目标实体抠出来
   im = out_map[..., roi[2]:(roi[4]+1), roi[1]:(roi[3]+1)]
   # print im.shape
   # 将抠出来的目标实体im，做adaptive_max_pool计算，最后得到一个固定的尺寸(7,7)== > (512, 7, 7)，方便后面进行批处理
   output.append(vgg.adaptive_max_pool(im)[0].data)
# ---------------------ROI Pooling


# ---------------------step_6: Classification 线性分类，预测预测框的类别，置信度和转为目标框的平移缩放系数（要与RPN区分）
# note: if your pytorch version is 0.3.1, you must run this:
# output = torch.stack(output)
output = torch.cat(output, 0)  # torch.Size([128, 512, 7, 7])
k = output.view(output.size(0), -1)  # [128, 25088]

k = torch.autograd.Variable(k)
k = vgg.roi_head_classifier(k)  # (128, 4096)
# torch.Size([128, 84])  84 ==> (20+1)*4,表示每个框有20个候选类别和一个置信度（假设为VOC数据集，共20分类），4表示坐标信息
pred_roi_locs = vgg.cls_loc(k)
# pred_roi_labels： [128, 21] 表示每个框的类别和置信度
pred_roi_labels = vgg.score(k)
print(pred_roi_locs.data.shape, pred_roi_labels.data.shape)  # torch.Size([128, 84]), torch.Size([128, 21])
# ---------------------Classification


# ---------------------step_7: 分类损失  (有效预测框真实系数与有效预测框的预测系数间损失，其中系数是转为目标框的坐标系数)
# 从上面step_4中，我们得到了预测框转为目标框的目标信息：
# 预测框的坐标系数(roi_locs)：  (128, 4)
# 预测框的所属类别(roi_labels)：(128, )

# 从上面step_6中，我们得到了预测框转为目标框的预测信息：
# 预测框的坐标系数：pred_roi_locs  (128, 84)
# 预测框的所属类别和置信度: pred_roi_labels  (128, 21)


gt_roi_loc = torch.from_numpy(roi_locs)
gt_roi_label = torch.from_numpy(np.float32(roi_labels)).long()
print(gt_roi_loc.shape, gt_roi_label.shape)  # torch.Size([128, 4]) torch.Size([128])

n_sample = pred_roi_locs.shape[0]
roi_loc = pred_roi_locs.view(n_sample, -1, 4)  # (128L, 21L, 4L)

roi_loc = roi_loc[torch.arange(0, n_sample).long(), gt_roi_label]  # 根据预测框的真实类别，找到真实类别所对应的坐标系数
# print(roi_loc.shape)  # torch.Size([128, 4])

roi_loss = vgg.roi_loss(roi_loc, pred_roi_labels, gt_roi_loc, gt_roi_label, weight=10.0)
print(roi_loss)  # 3.810348778963089


# 整体损失函数
total_loss = rpn_loss + roi_loss
print total_loss  # 5.149546355009079

SyntaxError: ignored

In [0]:
!rm simple_faster_rcnn.git


In [0]:
!wget https://github.com/liuyuemaicha/simple_faster_rcnn

--2020-02-09 02:19:07--  https://github.com/liuyuemaicha/simple_faster_rcnn
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘simple_faster_rcnn’

simple_faster_rcnn      [ <=>                ]  75.92K  --.-KB/s    in 0.008s  

2020-02-09 02:19:07 (9.28 MB/s) - ‘simple_faster_rcnn’ saved [77739]



In [0]:
cd ./simple_faster_rcnn

[Errno 20] Not a directory: './simple_faster_rcnn'
/content


torch.Size([1, 3, 800, 800])
[Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(512, 512, kernel_size=(3, 3),

TypeError: ignored

In [0]:
#coding:utf8

import torch
import torch.nn as nn
import torchvision
from PIL import Image, ImageDraw
import numpy as np


# cfg = {
#     'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
#     'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
#     'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
#     'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
# }

featur_cfg = ''


class VGG(nn.Module):

    def __init__(self):
        super(VGG, self).__init__()

        cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512]
        self.features = self._make_layers(cfg)
        self._rpn_model()

        size = (7, 7)
        self.adaptive_max_pool = torch.nn.AdaptiveMaxPool2d(size[0], size[1])
        self.roi_classifier()

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x

        # layers += [nn.Conv2d(in_channels, 512, kernel_size=3, padding=1)]
        return nn.Sequential(*layers)
        # return layers

    def _rpn_model(self, mid_channels=512, in_channels=512, n_anchor=9):
        self.rpn_conv = nn.Conv2d(in_channels, mid_channels, 3, 1, 1)
        self.reg_layer = nn.Conv2d(mid_channels, n_anchor * 4, 1, 1, 0)
        # I will be going to use softmax here. you can equally use sigmoid if u replace 2 with 1.
        self.cls_layer = nn.Conv2d(mid_channels, n_anchor * 2, 1, 1, 0)

        # conv sliding layer
        self.rpn_conv.weight.data.normal_(0, 0.01)
        self.rpn_conv.bias.data.zero_()

        # Regression layer
        self.reg_layer.weight.data.normal_(0, 0.01)
        self.reg_layer.bias.data.zero_()

        # classification layer
        self.cls_layer.weight.data.normal_(0, 0.01)
        self.cls_layer.bias.data.zero_()

    def forward(self, data):
        out_map = self.features(data)
        # for layer in self.features:
        #     # print layer
        #     data = layer(data)
        #     # print data.data.shape
        #
        # # out = data.view(data.size(0), -1)
        x = self.rpn_conv(out_map)
        pred_anchor_locs = self.reg_layer(x)  # 回归层，计算有效anchor转为目标框的四个系数
        pred_cls_scores = self.cls_layer(x)  # 分类层，判断该anchor是否可以捕获目标

        return out_map, pred_anchor_locs, pred_cls_scores

    def roi_classifier(self, class_num=20):  # 假设为VOC数据集，共20分类
        # 分类层
        self.roi_head_classifier = nn.Sequential(*[nn.Linear(25088, 4096),
                                                   nn.ReLU(),
                                                   nn.Linear(4096, 4096),
                                                   nn.ReLU()])
        self.cls_loc = nn.Linear(4096, (class_num+1) * 4)  # (VOC 20 classes + 1 background. Each will have 4 co-ordinates)
        self.cls_loc.weight.data.normal_(0, 0.01)
        self.cls_loc.bias.data.zero_()


        self.score = nn.Linear(4096, class_num+1)  # (VOC 20 classes + 1 background)

    def rpn_loss(self, rpn_loc, rpn_score, gt_rpn_loc, gt_rpn_label, weight=10.0):
        # 对与classification我们使用Cross Entropy损失
        gt_rpn_label = torch.autograd.Variable(gt_rpn_label.long())
        rpn_cls_loss = torch.nn.functional.cross_entropy(rpn_score, gt_rpn_label, ignore_index=-1)
        # print(rpn_cls_loss)  # Variable containing: 0.6931

        # 对于 Regression 我们使用smooth L1 损失
        pos = gt_rpn_label.data > 0  # Regression 损失也被应用在有正标签的边界区域中
        mask = pos.unsqueeze(1).expand_as(rpn_loc)
        # print(mask.shape)  # (22500L, 4L)

        # 现在取有正数标签的边界区域
        mask_pred_loc = rpn_loc[mask].view(-1, 4)
        mask_target_loc = gt_rpn_loc[mask].view(-1, 4)
        # print(mask_pred_loc.shape, mask_target_loc.shape)  # ((18L, 4L), (18L, 4L))

        # regression损失应用如下
        x = np.abs(mask_target_loc.numpy() - mask_pred_loc.data.numpy())
        # print x.shape  # (18, 4)
        # print (x < 1)
        rpn_loc_loss = ((x < 1) * 0.5 * x ** 2) + ((x >= 1) * (x - 0.5))
        # print rpn_loc_loss.shape  # (18, 4)
        rpn_loc_loss = rpn_loc_loss.sum()  # 1.1628926242031001
        # print rpn_loc_loss
        # print rpn_loc_loss.shape
        # rpn_loc_loss = np.squeeze(rpn_loc_loss)
        # print rpn_loc_loss

        N_reg = (gt_rpn_label > 0).float().sum()
        N_reg = np.squeeze(N_reg.data.numpy())

        # print "N_reg: {}, {}".format(N_reg, N_reg.shape)
        rpn_loc_loss = rpn_loc_loss / N_reg
        rpn_loc_loss = np.float32(rpn_loc_loss)
        # rpn_loc_loss = torch.autograd.Variable(torch.from_numpy(rpn_loc_loss))

        rpn_cls_loss = np.squeeze(rpn_cls_loss.data.numpy())
        # print "rpn_cls_loss: {}".format(rpn_cls_loss)  # 0.693146109581
        # print 'rpn_loc_loss: {}'.format(rpn_loc_loss)  # 0.0646051466465
        rpn_loss = rpn_cls_loss + (weight * rpn_loc_loss)
        # print("rpn_loss: {}".format(rpn_loss))  # 1.33919757605
        return rpn_loss

    def roi_loss(self, pre_loc, pre_conf, target_loc, target_conf, weight=10.0):
        # 分类损失
        target_conf = torch.autograd.Variable(target_conf.long())
        pred_conf_loss = torch.nn.functional.cross_entropy(pre_conf, target_conf, ignore_index=-1)
        # print(pred_conf_loss)  # Variable containing:  3.0515

        #  对于 Regression 我们使用smooth L1 损失
        # 用计算RPN网络回归损失的方法计算回归损失
        # pre_loc_loss = REGLoss(pre_loc, target_loc)
        pos = target_conf.data > 0  # Regression 损失也被应用在有正标签的边界区域中
        mask = pos.unsqueeze(1).expand_as(pre_loc)  # (128, 4L)

        # 现在取有正数标签的边界区域
        mask_pred_loc = pre_loc[mask].view(-1, 4)
        mask_target_loc = target_loc[mask].view(-1, 4)
        # print(mask_pred_loc.shape, mask_target_loc.shape)  # ((19L, 4L), (19L, 4L))

        x = np.abs(mask_target_loc.numpy() - mask_pred_loc.data.numpy())
        # print x.shape  # (19, 4)

        pre_loc_loss = ((x < 1) * 0.5 * x ** 2) + ((x >= 1) * (x - 0.5))
        # print(pre_loc_loss.sum())  # 1.4645805211187053

        N_reg = (target_conf > 0).float().sum()
        N_reg = np.squeeze(N_reg.data.numpy())
        pre_loc_loss = pre_loc_loss.sum() / N_reg
        pre_loc_loss = np.float32(pre_loc_loss)
        # print pre_loc_loss  # 0.077294916
        # pre_loc_loss = torch.autograd.Variable(torch.from_numpy(pre_loc_loss))
        # 损失总和
        pred_conf_loss = np.squeeze(pred_conf_loss.data.numpy())
        total_loss = pred_conf_loss + (weight * pre_loc_loss)

        return total_loss


if __name__ == '__main__':
    vgg = VGG()
    print vgg
    data = torch.randn((1, 3, 800, 800))
    print data.shape
    data = torch.autograd.Variable(data)
    out = vgg.forward(data)
    print out.data.shape


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

AttributeError: ignored